In [2]:
import pandas as pd
import sqlite3

### Шаг 1

(Балл - 2) Необходимо скачать CSV-файл - «transactions_for_dz2.csv», создать таблицу transaction_bd со всеми полями, загрузить данные из файла в таблицу и оставить таблицу со структурой:

(0, 'TRANSACTION_ID', 'INTEGER', 0, None, 0) - id транзакции
(1, 'TX_DATETIME', 'NUMERIC', 0, None, 0) - дата транзакции
(2, 'CUSTOMER_ID', 'INTEGER', 0, None, 0) - id клиента
(3, 'TERMINAL_ID', 'INTEGER', 0, None, 0) - id терминала
(4, 'TX_AMOUNT', 'REAL', 0, None, 0) - сумма транзакции

Также необходимо скачать второй CSV-файл - «client_info.csv», создать таблицу customer_bd со всеми полями, загрузить данные из файла в таблицу и получить таблицу со структурой:

(0, 'CLIENT_ID', 'INTEGER', 0, None, 0) - id клиента
(1, 'START_DT', 'NUMERIC', 0, None, 0) - дата начало записи о клиенте
(2, 'END_DT', 'NUMERIC', 0, None, 0) - дата закрытия записи о клиенте
(3, 'CLIENT_NAME', 'TEXT', 0, None, 0) - название клиента
(4, 'YEAR_BIRTH', 'TEXT', 0, None, 0) - дата рождение клиента

Примечание для следующих шагов: сумма транзакций не может быть null-ом, это всегда число.

In [3]:
transaction_df = pd.read_csv('./transactions_for_dz2.csv')
customer_df = pd.read_csv('./client_info.csv', sep=';')

In [4]:
transaction_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1048575 entries, 0 to 1048574
Data columns (total 5 columns):
 #   Column          Non-Null Count    Dtype  
---  ------          --------------    -----  
 0   TRANSACTION_ID  1048575 non-null  int64  
 1   TX_DATETIME     1048575 non-null  object 
 2   CUSTOMER_ID     1048575 non-null  int64  
 3   TERMINAL_ID     1048575 non-null  int64  
 4   TX_AMOUNT       1048575 non-null  float64
dtypes: float64(1), int64(3), object(1)
memory usage: 40.0+ MB


In [5]:
customer_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 4988 entries, 0 to 4987
Data columns (total 5 columns):
 #   Column       Non-Null Count  Dtype 
---  ------       --------------  ----- 
 0   START_DT     4988 non-null   object
 1   END_DT       4988 non-null   object
 2   CLIENT_NAME  4988 non-null   object
 3   YEAR_BIRTH   4988 non-null   int64 
 4   CLIENT_ID    4988 non-null   int64 
dtypes: int64(2), object(3)
memory usage: 195.0+ KB


In [8]:
#Создание таблиц
con = sqlite3.connect("./hw4.db")
sql_drop_transaction_bd = "DROP TABLE IF EXISTS transaction_bd;"
sql_drop_customer_bd = "DROP TABLE IF EXISTS customer_bd;"
sql_create_transaction_bd = """
CREATE TABLE transaction_bd (
    TRANSACTION_ID INTEGER NOT NULL
                           PRIMARY KEY,
    TX_DATETIME    NUMERIC NOT NULL,
    CUSTOMER_ID    INTEGER NOT NULL,
    TERMINAL_ID    INTEGER NOT NULL,
    TX_AMOUNT   REAL    NOT NULL
);
"""
sql_create_customer_bd = """
CREATE TABLE customer_bd (
    CLIENT_ID   INTEGER PRIMARY KEY
                        NOT NULL,
    START_DT    NUMERIC NOT NULL,
    END_DT      NUMERIC,
    CLIENT_NAME TEXT,
    YEAR_BIRTH  NUMERIC
);
"""
cur = con.cursor()
cur.execute(sql_drop_transaction_bd)
cur.execute(sql_create_transaction_bd)
cur.execute(sql_drop_customer_bd)
cur.execute(sql_create_customer_bd)
con.commit()

In [9]:
#Заполнение данными
sql_insert_transaction_bd = """
INSERT INTO transaction_bd (
                               TRANSACTION_ID,
                               TX_DATETIME,
                               CUSTOMER_ID,
                               TERMINAL_ID,
                               TX_AMOUNT
                           )
                           VALUES (
                               ?,
                               ?,
                               ?,
                               ?,
                               ?
                           );
"""
sql_insert_customer_bd = """
INSERT INTO customer_bd (
                            START_DT,
                            END_DT,
                            CLIENT_NAME,
                            YEAR_BIRTH,
                            CLIENT_ID
                        )
                        VALUES (
                            ?,
                            ?,
                            ?,
                            ?,
                            ?
                        );

"""
con.executemany(sql_insert_transaction_bd, transaction_df.values.tolist())
con.executemany(sql_insert_customer_bd, customer_df.values.tolist())
con.commit()


### Шаг 2

(Балл - 1 за каждый пункт) Написать скрипты:

In [11]:
# a. Вывести список всех клиентов, у которых между двумя ближайшими транзакциями
# был перерыв больше 35 дней хотя бы один раз 
# (клиент должен быть действующим, то есть дата закрытия записи о клиенте 
# не должна быть меньше дня относительно которого мы считаемся). 
# За дефолтную дату для выборки действующих клиентов брать ‘2023-05-01’
sql_query_a = """
with transction_interval as (
SELECT 
    tr.CUSTOMER_ID,
    JULIANDAY(tr.TX_DATETIME) - 
    JULIANDAY(LAG(tr.TX_DATETIME,1) OVER(PARTITION BY tr.CUSTOMER_ID ORDER BY tr.TX_DATETIME)) AS 'TX_INTERVAL'
FROM transaction_bd as tr
    join customer_bd as cus on cus.CLIENT_ID = tr.CUSTOMER_ID
WHERE cus.END_DT >= '2023-05-01')

SELECT distinct CUSTOMER_ID
FROM transction_interval
WHERE TX_INTERVAL > 35
"""
table_a = pd.read_sql(sql_query_a, con)
print(table_a)

    CUSTOMER_ID
0            24
1           480
2           639
3           707
4           812
5           896
6          1129
7          1299
8          1334
9          1459
10         1637
11         1923
12         2061
13         2093
14         2105
15         2201
16         2284
17         2307
18         2368
19         2374
20         3216
21         3352
22         3403
23         3418
24         3478
25         3608
26         3645
27         3990
28         4224
29         4388
30         4562
31         4618
32         4650
33         4875


In [12]:
# b. Вывести список клиентов, у которых максимальная сумма транзакции
# (имеется ввиду максимальное значение TX_AMOUNT) больше 50000.
# В ответе должен быть 
#   id клиента,
#   максимальное значение суммы транзакций,
#   минимальное значение суммы транзакций 
#(клиент должен быть действующим, то есть дата закрытия записи о клиенте не должна быть меньше дня относительно которого мы считаемся)
# За дефолтную дату для выборки действующих клиентов брать ‘2023-05-01’
sql_query_b = """
SELECT tr.CUSTOMER_ID,
       MAX(tr.TX_AMOUNT) AS MAX_AMOUNT,
       MIN(tr.TX_AMOUNT) AS MIN_AMOUNT
FROM transaction_bd as tr
    join customer_bd as cus on cus.CLIENT_ID = tr.CUSTOMER_ID
WHERE cus.END_DT >= '2023-05-01'
GROUP BY tr.CUSTOMER_ID
HAVING MAX(tr.TX_AMOUNT) > 50000;
"""
table_b = pd.read_sql(sql_query_b, con)
print(table_b)



   CUSTOMER_ID  MAX_AMOUNT  MIN_AMOUNT
0         3013    77212.50        1.46
1         3494    53213.00       85.76
2         4253    51937.25       37.95


In [14]:
# c. Вывести топ-10 клиентов (сортируя их по убыванию id),
# у которых сумма дневных транзакций больше суммы вечерних транзакций
# Условимся, что дневными считаются транзакции, которые были 
# произведены до 12 часов, а вечерними - после 12 часов включительно.
# (клиент должен быть действующим, то есть дата закрытия записи о клиенте 
# не должна быть меньше дня относительно которого мы считаемся).
# За дефолтную дату для выборки действующих клиентов брать ‘2023-05-01’
sql_query_c = """
SELECT CUSTOMER_ID
  FROM (
           SELECT CUSTOMER_ID,
                  LAG(SUM_TX_AMOUNT, 1) OVER (PARTITION BY CUSTOMER_ID ORDER BY DAY_PART) - SUM_TX_AMOUNT AS DIFF
             FROM (
                      SELECT CUSTOMER_ID,
                             CASE WHEN CAST (strftime("%H", TX_DATETIME) AS INTEGER) < 12 THEN "AM" ELSE "PM" END AS DAY_PART,
                             SUM(TX_AMOUNT) AS SUM_TX_AMOUNT
                        FROM transaction_bd AS tr
                             JOIN
                             customer_bd AS cus ON cus.CLIENT_ID = tr.CUSTOMER_ID
                       WHERE cus.END_DT >= '2023-05-01'
                       GROUP BY CUSTOMER_ID,
                                CASE WHEN CAST (strftime("%H", TX_DATETIME) AS INTEGER) < 12 THEN "AM" ELSE "PM" END
                  )
       )
 WHERE DIFF > 0
 ORDER BY CUSTOMER_ID DESC
 LIMIT 10;
"""
table_c = pd.read_sql(sql_query_c, con)
print(table_c)

   CUSTOMER_ID
0         4997
1         4993
2         4990
3         4987
4         4983
5         4982
6         4978
7         4976
8         4973
9         4972


In [17]:
# d. Найти для каждого клиента день, когда у него была максимальная сумма транзакции, 
# то есть вывести клиента, дату транзакции, сумму транзакции (клиент должен быть действующим,
# то есть дата закрытия записи о клиенте не должна быть меньше дня относительно которого мы считаемся).
# За дефолтную дату для выборки действующих клиентов брать ‘2023-05-01’ 
sql_query_d = """
SELECT CUSTOMER_ID,
       TX_DATE,
       MAX(SUM_TX_AMOUNT) AS MAX_SUM_TX_AMOUNT
  FROM (
           SELECT tr.CUSTOMER_ID,
                  date(tr.TX_DATETIME) AS TX_DATE,
                  SUM(tr.TX_AMOUNT) AS SUM_TX_AMOUNT
             FROM transaction_bd AS tr
                  JOIN
                  customer_bd AS cus ON cus.CLIENT_ID = tr.CUSTOMER_ID
            WHERE cus.END_DT >= '2023-05-01'
            GROUP BY tr.CUSTOMER_ID,
                     date(tr.TX_DATETIME) 
       )
 GROUP BY CUSTOMER_ID;
"""
table_d = pd.read_sql(sql_query_d, con)
print(table_d)

      CUSTOMER_ID     TX_DATE  MAX_SUM_TX_AMOUNT
0               0  2023-02-01            4084.94
1               1  2023-01-28           19318.16
2               2  2023-04-14            6190.41
3               5  2023-03-17            4906.86
4               6  2023-04-12            1504.53
...           ...         ...                ...
3731         4991  2023-01-09            3030.25
3732         4992  2023-03-01            2733.89
3733         4993  2023-02-21            2953.64
3734         4996  2023-01-25             784.33
3735         4997  2023-02-04            3733.15

[3736 rows x 3 columns]


In [21]:
# e. Вычислить интегральное распределение суммы транзакций для каждого дня за весь период,
# то есть вывести дату транзакции,
# сумму транзакции и
# интегральное распределение по сумме (не учитываем что клиент может быть недействующим)
sql_query_e = """
SELECT TX_DATE,
       SUM_TX_AMOUNT,
       CUME_DIST() OVER (ORDER BY TX_DATE) AS Cume_Dist
  FROM (
           SELECT DATE(TX_DATETIME) AS TX_DATE,
                  SUM(TX_AMOUNT) AS SUM_TX_AMOUNT
             FROM transaction_bd
            GROUP BY DATE(TX_DATETIME) 
       );
"""
table_e = pd.read_sql(sql_query_e, con)
print(table_e)



        TX_DATE  SUM_TX_AMOUNT  Cume_Dist
0    2023-01-01     4827656.26   0.009091
1    2023-01-02     4862551.41   0.018182
2    2023-01-03     5058973.71   0.027273
3    2023-01-04     4938142.47   0.036364
4    2023-01-05     5002954.23   0.045455
..          ...            ...        ...
105  2023-04-16     5299386.81   0.963636
106  2023-04-17     5100973.31   0.972727
107  2023-04-18     5233557.39   0.981818
108  2023-04-19     5194846.44   0.990909
109  2023-04-20     1795235.72   1.000000

[110 rows x 3 columns]
